# Image

In [5]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from os import listdir


from keras.models import Sequential
from keras.models import Model
from keras.optimizers import Adam
from keras import preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D,Flatten, Dense, Dropout

#import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)


#import logging
##logger = logging.getLogger()
#logger.setLevel(logging.CRITICAL)

import warnings
warnings.simplefilter(action='ignore', category=Warning)

import os

#Path-----

path = "new_img/"
list_photos = [file for file in listdir(path)]
print(len(list_photos))

import time
tempsX = time.time()
def _time_(x):
    duration1 = time.time()-tempsX
    return print("temps de traitement "+ str(x) + " : ",  round(duration1/60,2), "minutes")


mA = pd.read_csv('mDataImgName.csv')
mB = pd.read_csv('mDataImgNumm.csv')
mC = pd.read_csv('mDataImgCat.csv')

df = mA.join(mB)
df = df.join(mC)
df = df.rename(columns={'img_name': 'id','Img_Cat_' : 'label', 'img_Num' : 'num'})

model_savepath    = 'cnn_vgg16_model_trained_2.h5'     

TRAINING_DIR      = 'img_train_/'
TESTING_DIR       = 'img_test_/'

IMGSIZE       = 224    # Taille de l'image en input
EPOCH         = 22     # nombre d'epoch 
BATCH_SIZE    = 16     # traitement par batch d'images avant la descente de gradient
FREEZE_LAYERS = 15     # pour un VGG16 freeze de réapprentissage de certaines couches
TRAIN         = True 

_time_("#IMG 1 - model_savepath")

_count = df.num < 210
df_test = df[_count]
count_ = df.num >= 210
df_train = df[ count_]

_time_("#IMG 2 - Tes-Train CNN ")

train_datagen =  ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.20)
_time_("#IMG 3 - train_datagen")

train_generator = train_datagen.flow_from_dataframe(
            df_train,
            TRAINING_DIR,
            x_col='id',
            y_col='label',
            has_ext=True,
            shuffle=True,
            target_size=(224, 224),
            batch_size=BATCH_SIZE,
            subset='training',
            class_mode='categorical')

_time_("#IMG 4 - train_generator")


from keras.utils.np_utils import to_categorical
train_labels = to_categorical(train_generator.classes)
_time_("#IMG 5 - train_labels")

valid_generator = \
        train_datagen.flow_from_dataframe(
            df_train,
            TRAINING_DIR,
            x_col='id',
            y_col='label',
            has_ext=True,
            shuffle=True,
            target_size=(224, 224),
            batch_size=BATCH_SIZE,
            subset='validation',
            class_mode='categorical')

_time_("#IMG 6 - valid_generator")

test_files = os.listdir(TESTING_DIR)
df_test = pd.DataFrame({"id": test_files, 'label': 'nan'})
print(df_test.shape)

_time_("#IMG 7 - df_test")

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)    

_time_("#IMG 8 - test_datagen")

test_generator = \
    test_datagen.flow_from_dataframe(
        df_test, 
        TESTING_DIR, 
        x_col='id',
        y_col=None, 
        has_ext=True, 
        target_size=(IMGSIZE, IMGSIZE), 
        class_mode=None, 
        seed=42,
        batch_size=1, 
        shuffle=False  )

_time_("#IMG 9 - test_generator")



from keras.applications import ResNet50
base_model = Sequential()
base_model.add(ResNet50(include_top=False, pooling='avg'))
base_model.add(Dense(7, activation='softmax'))
base_model.layers[0].trainable = False
base_model.summary()

_time_("#IMG 10 - test_generator")

1051
temps de traitement #IMG 1 - model_savepath :  0.0 minutes
temps de traitement #IMG 2 - Tes-Train CNN  :  0.0 minutes
temps de traitement #IMG 3 - train_datagen :  0.0 minutes
Found 672 validated image filenames belonging to 7 classes.
temps de traitement #IMG 4 - train_generator :  0.0 minutes
temps de traitement #IMG 5 - train_labels :  0.0 minutes
Found 168 validated image filenames belonging to 7 classes.
temps de traitement #IMG 6 - valid_generator :  0.0 minutes
(211, 2)
temps de traitement #IMG 7 - df_test :  0.0 minutes
temps de traitement #IMG 8 - test_datagen :  0.0 minutes
Found 210 validated image filenames.
temps de traitement #IMG 9 - test_generator :  0.0 minutes


--- Logging error ---
Traceback (most recent call last):
  File "/opt/anaconda3/envs/baseNew/lib/python3.8/logging/__init__.py", line 1081, in emit
    msg = self.format(record)
  File "/opt/anaconda3/envs/baseNew/lib/python3.8/logging/__init__.py", line 925, in format
    return fmt.format(record)
  File "/opt/anaconda3/envs/baseNew/lib/python3.8/logging/__init__.py", line 664, in format
    record.message = record.getMessage()
  File "/opt/anaconda3/envs/baseNew/lib/python3.8/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/anaconda3/envs/baseNew/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/envs/baseNew/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/opt/anaconda3/envs/baseNew/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.

ImportError: cannot import name 'ResNet50' from 'keras.applications' (/opt/anaconda3/envs/baseNew/lib/python3.8/site-packages/keras/applications/__init__.py)

In [ ]:
#Test trouve DataFrame

In [ ]:

base_model.compile(loss="categorical_crossentropy",
              optimizer="Adam",
              metrics=['acc'])



_time_("#IMG 11 - base_model")

In [ ]:
base_model.fit_generator(
                 generator=train_generator,
                 steps_per_epoch = 30,
                 validation_data = valid_generator,
                 validation_steps = 10,
                 epochs = 5)
     
_time_("#IMG 12 - history")

In [ ]:
# Evaluation du modèle
(eval_loss, eval_accuracy) = base_model.evaluate_generator(generator=valid_generator, steps=30)
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))
print("[INFO] Loss: {}".format(eval_loss))

_time_("#IMG 13 - Evaluation")


#Test Predict

In [ ]:
image = tf.keras.preprocessing.image.load_img('/Users/rogerrabbit/Desktop/Projet 6 Files/new_img/10-Watches.jpg')
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  # Convert single image to a batch.
print(input_arr.shape)


In [ ]:
lst_pred = []
st = '/Users/rogerrabbit/Desktop/Projet 6 Files/new_img/'
for i in range (90,95):
    image = tf.keras.preprocessing.image.load_img(st+ str(i) + '-Home_Decor_Festive_Needs.jpg',target_size=(224,224))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    predictions = base_model.predict(input_arr)
    predictions = pd.DataFrame(predictions)
    print(predictions)
    
print("=================================")
    
lst_pred = []
st = '/Users/rogerrabbit/Desktop/Projet 6 Files/new_img/'
for i in range (5,15):
    image = tf.keras.preprocessing.image.load_img(st+ str(i) + '-Watches.jpg',target_size=(224,224))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    predictions = base_model.predict(input_arr)
    predictions = pd.DataFrame(predictions)
    print(predictions)

In [ ]:
st = '/Users/rogerrabbit/Desktop/Projet 6 Files/new_img/9-Watches.jpg'

image = tf.keras.preprocessing.image.load_img(st,target_size=(224,224))
input_arr = tf.keras.preprocessing.image.img_to_array(image)
input_arr_ = np.array([input_arr])  # Convert single image to a batch.
predictions = base_model.predict(input_arr_)
predictions = pd.DataFrame(predictions)
print(input_arr_.shape)
print(input_arr.shape)

train_generator



_labels

# TXT

In [ ]:
# Base
from IPython.display import Audio
import warnings
from tqdm import tqdm
import numpy as np
import pandas as pd
# Graph
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# Loop compteur
# Warning
warnings.simplefilter(action='ignore', category=Warning)
# Notif audio
import time
tempsX = time.time()
def _time_(x):
    duration1 = time.time()-tempsX
    return print("temps de traitement "+ str(x) + " : ",  round(duration1/60,2), "minutes")

def mFinito():
    display(Audio('/Users/soso/Downloads/son_multiplex_canal_plus.mp3', autoplay=True))

from keras.preprocessing.sequence import pad_sequences
# Notif Chrome
#%load_ext jupyternotify
# Largeur colonne
#pd.set_option('max_colwidth', 200)
# df create

from sklearn.model_selection import train_test_split

def _df_(x):
    return pd.DataFrame(x)

#Add
import gensim
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn import metrics
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import gensim
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:



X = pd.read_csv('m_x_df_TXT.csv')
y = pd.read_csv('m_y_df_TXT.csv')
print(X.shape)
print(y.shape)
_time_("#TXT 1 - import_img_")

_time_("#TXT 2 - _create_X_")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=21)

print('Shape of training data: ')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data: ')
print(X_test.shape)
print(y_test.shape)

_time_("#TXT 3 - train_test_split")

import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import LSTM, SpatialDropout1D


# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100


print('X.shape[1]------', X.shape[1])
model = tf.keras.Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_time_("#TXT 4 - compile")

epochs = 10
batch_size = 64


history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)

_time_("#TXT 5 - model.fit(")



scores = model.evaluate(X_test, y_test, verbose=0)
# Displays the accuracy of correct sentiment prediction over test data
print("Accuracy: %.2f%%" % (scores[1]*100))

_time_("#TXT 6 - model.evaluate")

In [ ]:


scores = model.evaluate(X_test, y_test, verbose=0)
# Displays the accuracy of correct sentiment prediction over test data
print("Accuracy: %.2f%%" % (scores[1]*100))

_time_("#TXT 6 - model.evaluate")

In [ ]:
#test



In [ ]:
X = pd.read_csv('m_x_df_TXT.csv')

In [ ]:
#from keras.layers import Input
#from keras import layers
#tag_input = Input(shape=(None,), dtype='int32', name='tag')
#embedded_tag = layers.Embedding(50000, 100)(tag_input)
#encoded_tag = layers.LSTM(256)(embedded_tag)

# Reunion

In [ ]:
a = tf.keras.applications.resnet50.ResNet50(
    include_top=True, weights='imagenet', input_tensor=None,
    input_shape=None, pooling=None, classes=1000)

a.summary()

In [ ]:
from tqdm import tqdm
mA = pd.read_csv('mDataImgName.csv')
mB = pd.read_csv('mDataImgLab.csv')
mC = pd.read_csv('mDataImgCat.csv')
data = mA.join(mB)
data = data.join(mC)
data = data.rename(columns={'img_name': 'image_path','Img_Cat_label' : 'label', 'Img_Cat_' : 'label_name'})
lst_img = []


for xXx in tqdm(data.image_path):
    image = tf.keras.preprocessing.image.load_img("new_img/" + str(xXx) ,target_size=(224,224))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    lst_img.append(input_arr)

In [ ]:
lst_img = np.array(lst_img)
lst_img.shape

In [ ]:
lst_txt = pd.read_csv('m_x_df_TXT.csv')
lst_label = pd.read_csv('m_y_df_TXT.csv')
print(X.shape)
print(y.shape)

In [ ]:
lst_txt = lst_txt.to_numpy()
lst_txt

In [ ]:
from sklearn.model_selection import train_test_split

X_tR_txt, X_tE_txt, y_tR_txt, y_tE_txt = train_test_split(lst_txt, lst_label, test_size=0.3,random_state=21)

X_tR_img, X_tE_img, y_tR_img, y_tE_img = train_test_split(lst_img, lst_label, test_size=0.3,random_state=21)

In [ ]:
tag_input = Input(shape=(None,), dtype='int32', name='tag')
m_txt = Embedding(5000, 100)(tag_input)
m_txt = LSTM(512)(m_txt)
m_txt = Dense(7, activation='relu')(m_txt)
m_txt.summary

In [ ]:
concatenated = concatenate([m_txt, m_img], axis=-1)
output = Dense(7, activation='sigmoid')(concatenated)
model = Model([image_input, tag_input], output)
model.summary()

In [ ]:
print(X_tR_img.shape)
print(X_tR_txt.shape)
print(y_tE_txt.shape)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc']) 
model.fit([X_tR_img, X_tR_txt], y_tE_txt, epochs=30, batch_size=64)



# ------ TEST -------

In [ ]:
def base_model(input1, input2):
   
    input1 = np.expand_dims(input1,1)
    input2 = np.expand_dims(input2, 1)

    # Define Input Layers for ANN
    input1 = Input(shape = (input1.shape[1],), name = "input1")
    input2 = Input(shape = (input2.shape[1],), name = "input2")

    inputs = [input1, input2]

    # First Branch of DNN (input1)
    x = BatchNormalization()(input1) 
    x = Dense(units = 5, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)

    # Second Branch of DNN (input2)
    y = BatchNormalization()(input2)  
    y = Dense(units = 256, activation = "relu", kernel_regularizer=regularizers.l2(0.01))(y)
    
    #Third Branch of DNN (Detergents)
    z = BatchNormalization()(input3) 
    z = Dense(units = 5, activation='relu', kernel_regularizer=regularizers.l2(0.01))(z)

    # Merge the input models into a single large vector
    concatenated = Concatenate()([x, y, z])
    
    #Apply Final Output Layer
    outputs = Dense(32, activation = "relu",  kernel_regularizer=regularizers.l2(0.01))(concatenated)
    outputs = Dropout(0.08)(outputs)
    outputs = Dense(1, name = "output")(outputs)

    # Create an Interpretation Model (Accepts the inputs from previous branches and has single output)
    model = Model(inputs = inputs, outputs = outputs)
    
    # Compile the Model
    model.compile(loss='mse', optimizer = Adam(lr = 0.00001), metrics = ['mse'])

    # Summarize the Model Summary
    model.summary()
    
    return model

In [ ]:
import tensorflow as tf
#base_model = tf.keras.Sequential()
from keras.applications.resnet50 import ResNet50
from tensorflow import keras
from tensorflow.keras import layers


base_model = ResNet50(include_top=False, pooling='avg')
conv1 = Conv2D(32, (3,3), activation = 'relu')(base_model.output)
pool1 = MaxPooling2D(2,2)(conv1)
bn1 = BatchNormalization(axis=chanDim)(pool1)
drop1 = Dropout(0.2)(bn1)
flatten1 = Flatten()(drop1)
fc2 = Dense(classes, activation='softmax')(flatten1)
base_model = Model(inputs=base_model.input, outputs=fc2)
base_model.summary()





_time_("#IMG 10 - test_generator")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

base_model = tf.keras.Sequential()

emb = layers.Embedding(5000, 1000, input_length=10)
spac = layers.SpatialDropout1D(0.2)
lstm = layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2)
dens = layers.Dense(7, activation='softmax')

base_model = tf.keras.Sequential()
base_model.add(emb)
base_model.add(spac)
base_model.add(lstm)
base_model.add(dens)


base_model.summary()

In [ ]:
concatenated = concatenate([model, base_model])

In [ ]:
from keras.applications import ResNet50
from keras.layers import Embedding, SpatialDropout1D, LSTM,Conv2D, Dense
model = Sequential()
model.add(Embedding(5000, 1000, input_length=10))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='softmax'))
model.summary()

In [ ]:
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers import Dense, Input

model1_in = Input(shape=(27, 27, 1))
model1_out = Dense(300, input_dim=40, activation='relu', name='layer_1')(model1_in)
model1 = Model(model1_in, model1_out)

model2_in = Input(shape=(27, 27, 1))
model2_out = Dense(300, input_dim=40, activation='relu', name='layer_2')(model2_in)
model2 = Model(model2_in, model2_out)


concatenated = concatenate([model1_out, model2_out])
out = Dense(1, activation='softmax', name='output_layer')(concatenated)

merged_model = Model([model1_in, model2_in], out)
merged_model.compile(loss='binary_crossentropy', optimizer='adam', 
metrics=['accuracy'])

checkpoint = ModelCheckpoint('weights.h5', monitor='val_acc',
save_best_only=True, verbose=2)
early_stopping = EarlyStopping(monitor="val_loss", patience=5)

merged_model.fit([x1, x2], y=y, batch_size=384, epochs=200,
             verbose=1, validation_split=0.1, shuffle=True, 
callbacks=[early_stopping, checkpoint])

In [ ]:
#scores = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
#Text

X = pd.read_csv('m_x_df_TXT.csv')
y = pd.read_csv('m_y_df_TXT.csv')

In [ ]:
import tensorflow as tf
from tensorflow import keras

num_classes = 10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


encoder_input = tf.keras.layers.Input(shape=(32, 32, 3), name="input")
x = tf.keras.layers.Conv2D(16, 3,activation = 'relu', kernel_initializer = keras.initializers.RandomUniform)(encoder_input)
x = tf.keras.layers.Conv2D(32, 3, activation = 'relu')(x)
x = tf.keras.layers.MaxPooling2D(3)(x)
x = tf.keras.layers.Conv2D(32, 3,activation = 'relu')(x)
x = tf.keras.layers.Conv2D(16, 3, activation = 'relu')(x)
encoder_output = tf.keras.layers.GlobalMaxPooling2D()(x)

encoder = tf.keras.Model(inputs=encoder_input, outputs=encoder_output, name = 'encoder')
encoder.summary()

#Decoder
decoder_input = tf.keras.layers.Reshape((4, 4, 1))(encoder_output)
x = tf.keras.layers.Conv2DTranspose(16, 3, activation = 'relu')(decoder_input)
x = tf.keras.layers.Conv2DTranspose(32, 3, activation = 'relu')(x)
x = tf.keras.layers.UpSampling2D(3)(x)
x = tf.keras.layers.Conv2DTranspose(16, 3, activation = 'relu')(x)
decoder_output = tf.keras.layers.Conv2DTranspose(1, 3, activation = 'relu')(x)


autoencoder = keras.Model(inputs = encoder_input, outputs = decoder_output, name='autoencoder')
autoencoder.summary()

In [ ]:
from keras.layers import Dense
from keras.applications import ResNet50
base_model = Sequential()
base_model.add(ResNet50(include_top=False, pooling='avg'))
base_model.add(Dense(7, activation='softmax'))
base_model.layers[0].trainable = False
base_model.summary()

In [ ]:
from keras.layers import Embedding, SpatialDropout1D, LSTM,Conv2D, Dense
model = Sequential()
model.add(Embedding(5000, 100, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='softmax'))
model.summary()

In [ ]:
#Model A

from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers import Dense, Input

model1_in = Input(shape=(224, 224, 3))
model1_in = (ResNet50(include_top=False, pooling='avg'))(model1_in)
model1_out = Dense(7, input_dim=40, activation='relu', name='layer_1')(model1_in)
model1_out
model1_out.summary()

In [ ]:


model2_in = Input(shape=(None, 10))
model2_out = Dense(300, input_dim=40, activation='relu', name='layer_2')(model2_in)
model2 = Model(model2_in, model2_out)


concatenated = concatenate([model1_out, model2_out])
out = Dense(1, activation='softmax', name='output_layer')(concatenated)

merged_model = Model([model1_in, model2_in], out)
merged_model.summary()


In [ ]:
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers import Dense, Input

model1_in = Input(shape=((224, 224, 3))
model1_out = Dense(300, input_dim=40, activation='relu', name='layer_1')(model1_in)
model1 = Model(model1_in, model1_out)

model2_in = Input(shape=(27, 27, 1))
model2_out = Dense(300, input_dim=40, activation='relu', name='layer_2')(model2_in)
model2 = Model(model2_in, model2_out)


concatenated = concatenate([model1_out, model2_out])
out = Dense(1, activation='softmax', name='output_layer')(concatenated)

merged_model = Model([model1_in, model2_in], out)
merged_model.summary()


In [ ]:
concatenated = layers.concatenate([x, encoded_tag], axis=-1)
output = layers.Dense(1, activation='sigmoid')(concatenated)model = Model([image_input, tag_input], output)
model.summary()